In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-20 23:30:07--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.70MB/s    in 0.2s    

2022-12-20 23:30:08 (5.70 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)


In [5]:
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...|2013-01-14 00:01:00|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...| 

In [6]:
# Get the number of rows in the DataFrame.
df.count()

5906333

In [7]:
df = df.dropna()
print(df.count())

5905269


In [8]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



# Transform the Data

## Create the "review_id_table".

In [9]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1KKOXHNI8MSXU|   32158956|B01KL6O72Y|      24485154|2013-01-14 00:01:00|
|R26SP2OPDK4HT7|    2714559|B01ID3ZS5W|     363128556|2014-01-04 00:03:00|
| RWQEDYAX373I1|   12608825|B01I497BGY|     811958549|2015-01-12 00:07:00|
|R231YI7R4GPF6J|   25482800|B01HDXFZK6|     692205728|2015-01-03 00:06:00|
|R3KO3W45DD0L1K|    9310286|B01G6MBEBY|     431150422|2015-01-12 00:06:00|
|R1C4QH63NFL5NJ|   26631939|B01FWRXN0Y|     366144407|2014-01-22 00:04:00|
|R2GP65O1U9N7BP|   48785098|B01EXNH1HE|     786052021|2015-01-28 00:07:00|
|R3O29CT5MQQ3XQ|   39548589|B01E7OL09O|     108920964|2015-01-10 00:07:00|
|R1ZECD2AA8QFF6|   29355866|B01DXHX81O|     317132458|2015-01-09 00:08:00|
|R2S79GCF6J89OA|   27477484|B01DDULIJK|     110598191|2014-01-24 00:05:00|
|R1CBCUGNP37MPK|   176858

In [10]:
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create the "products" Table

In [11]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01427DDXM|Zoozie LA Women's...|
|B013TPITFA|Tabeez Women's Pl...|
|B013FU4Y9E|Ekouaer Women's U...|
|B013FNK6PC|Ezcosplay Women O...|
|B013FACXVK|Womens Two Piece ...|
|B013DHKQ3W|OURS Women's 3 4 ...|
|B0131D4NKU|Genie Women's Jeg...|
|B0131CA9K4|BeWild Brand - Ju...|
|B0130ZDFTE|Flora&Fred Women'...|
|B012Y665WY|AW Fashion's I Ne...|
|B012X0G1KW|KAMACO Women's Se...|
|B012VOPU84|iLoveSIA Women's ...|
|B012UUEWZG|Harley-Davidson W...|
|B012LQU4CE|Women's Laced & L...|
|B012ERJSKO|Amoin Style Women...|
|B012ADLW92|Cocoship Retro 50...|
|B012980QUY|Sanwood® Cute Bab...|
|B0125V8SZK|OURS Women's Comf...|
|B0125M6TZ0|Modern Kiwi Solid...|
|B01254G39K|Alberto Cardinali...|
+----------+--------------------+
only showing top 20 rows



In [12]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



## Create the "customers" Table

In [13]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50612720|           559|
|   33883540|           351|
|   33924372|           344|
|   37474421|           282|
|   15006109|           263|
|   45825821|           262|
|    7996182|           260|
|   32254988|           254|
|   12919673|           252|
|   40333751|           249|
|   12824778|           248|
|   22985631|           198|
|   10558864|           194|
|    6724212|           179|
|   45547332|           178|
|   36217485|           175|
|    9652516|           173|
|   45187592|           171|
|    6534413|           160|
|   14230771|           155|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [14]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])
vine_table_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|
|R26SP2OPDK4HT7|          5|            1|          2|   N|
| RWQEDYAX373I1|          5|            0|          0|   N|
|R231YI7R4GPF6J|          5|            0|          0|   N|
|R3KO3W45DD0L1K|          5|            0|          0|   N|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|
|R2GP65O1U9N7BP|          5|            0|          0|   N|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|
|R2S79GCF6J89OA|          3|            0|          0|   N|
|R1CBCUGNP37MPK|          5|            0|          0|   N|
|R3NU4WUR5ZNV1V|          5|            0|          0|   N|
|R32EPCJ3XF8LGE|          5|            0|          0|   N|
|R1XIBC6WQBW31M|          5|            

# Load

In [17]:
mode = "append"
jdbc_url="jdbc:postgresql://cloud-etl-ss.caxkitwqsdzv.us-west-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "postgres", "driver":"org.postgresql.Driver"}

In [18]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write vine_df to table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored